In [14]:
tour = [1, 40, 37, 34, 33, 30, 21, 20, 19, 18, 16, 15, 13, 17, 22, 12, 24, 10, 8]

In [15]:
import numpy as np
rnd = np.random
import math
import matplotlib.pyplot as plt
from docplex.mp.model import Model
import tsplib95 
import networkx as nx
import pandas as pd
import warnings
from itertools import combinations
warnings.filterwarnings("ignore", category=Warning)
problem = tsplib95.load('dantzig42.tsp')
G = problem.get_graph()

In [16]:
import time

start_time = time.time()
def find_worst_edge(tour):
    seq_best_edge = pd.DataFrame(columns=['V1','V1+1','Distance'])
    
    for i in range(len(tour)):
        v1 = tour[i]
        v1p1 = tour[(i + 1) % len(tour)]
        edge_dist = problem.get_weight(v1,v1p1)
        seq_best_edge = seq_best_edge.append({'V1': v1, 'V1+1': v1p1, 'Distance': edge_dist}, ignore_index=True)
        
    seq_best_edge = seq_best_edge.sort_values('Distance', ascending=False).reset_index(drop=True)
    #do the iter by the liist
    for idx, col in seq_best_edge.iterrows():
        edge_start = seq_best_edge['V1'][idx]
        edge_end = seq_best_edge['V1+1'][idx]
        worst_edge_index = tour.index(edge_start)
#         print('ok', worst_edge_index)
        
        for i in range(len(tour)):
            if i != worst_edge_index and i != (worst_edge_index + 1) % len(tour): 
                new_tour = tour[:]
#                 print(new_tour)
                
                new_tour[worst_edge_index + 1 : i + 1] = reversed(tour[worst_edge_index + 1 : i + 1])
                
                old_dist = problem.get_weight(tour[worst_edge_index], tour[(worst_edge_index + 1) % len(tour)]) \
                            + problem.get_weight(tour[i], tour[(i + 1) % len(tour)])
                new_dist = problem.get_weight(new_tour[worst_edge_index], new_tour[(worst_edge_index + 1) % len(new_tour)]) \
                            + problem.get_weight(new_tour[i], new_tour[(i + 1) % len(new_tour)])
                
                if new_dist < old_dist:
                    return new_tour
                    print(new_tour)
                    
    return tour

def process(tour):
    while True:
        new_tour = find_worst_edge(tour)
        if new_tour == tour:
            distance = sum(problem.get_weight(new_tour[i],new_tour[i+1]) for i in range(len(new_tour)-1))
            return tour
        else:
            tour = new_tour
            continue

new = process(tour)
end_time = time.time()
print(f"Runtime of the code is {end_time - start_time} seconds")
print(new)


Runtime of the code is 0.07603573799133301 seconds
[1, 40, 37, 34, 33, 30, 21, 20, 19, 18, 16, 15, 13, 17, 22, 24, 12, 10, 8]


Greedy [1, 2, 5, 4, 3, 8, 9, 10, 12, 11, 24, 27, 26, 25, 7, 6, 31, 30, 28, 22, 23, 13, 14, 15, 16, 17, 18, 19, 20, 21, 29, 33, 32, 34, 35, 36, 37, 38, 39, 40, 41, 42]
NIH [1, 42, 41, 40, 5, 4, 3, 9, 10, 12, 11, 23, 22, 17, 16, 13, 14, 15, 18, 19, 20, 21, 29, 28, 26, 27, 24, 25, 8, 7, 6, 31, 30, 32, 33, 34, 35, 36, 37, 38, 39, 2]
NNH [1, 2, 3, 4, 5, 6, 30, 31, 7, 8, 9, 10, 25, 26, 27, 24, 11, 12, 13, 14, 15, 16, 18, 19, 20, 17, 23, 22, 21, 28, 29, 33, 32, 34, 35, 36, 37, 38, 39, 40, 41, 42]
